In [1]:
import mysql.connector as sql
import pandas as pd
import datetime as dt
import numpy as np
from sorted_months_weekdays import *
from sort_dataframeby_monthorweek import *
import matplotlib.pyplot as plt
import math
import json
from pandas.io.json import json_normalize

In [25]:
db_connection = sql.connect(host='localhost', database='collection_visits', user='pandas', password='password')

visits = pd.read_sql('SELECT * FROM Visits', con=db_connection)
visits['created'] = pd.to_datetime(visits['created'], unit='s')
visits['lastmodified'] = pd.to_datetime(visits['lastmodified'], unit='s')
visits['registered'] = visits['registered'].replace(0, np.nan)
visits['registered'] = pd.to_datetime(visits['registered'], unit='s')
visits['deleted'] = visits['deleted'].replace(0, np.nan)
visits['deleted'] = pd.to_datetime(visits['deleted'], unit='s')
visits['paired'] = visits['paired'].replace(0, np.nan)
visits['paired'] = pd.to_datetime(visits['paired'], unit='s')
visits['first_accessed'] = pd.to_datetime(visits['first_accessed'], unit='s')
# visits['created_ip'].replace(0, np.nan, inplace=True)
# visits['registered_ip'].replace(0, np.nan, inplace=True)
# visits['paired_ip'].replace(0, np.nan, inplace=True)
# visits['processed'].replace(0, np.nan, inplace=True)
visits['exported'].replace(0, np.nan, inplace=True)
visits['title'].fillna(value=np.nan, inplace=True)
visits['note'].replace(r'^\s*$', np.NaN, regex=True, inplace=True)

In [26]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 50)
pd.set_option("max_colwidth", 1000)

In [28]:
visits

,id,shortcode,user_id,created,lastmodified,registered,expires,paired,ticket,deleted,created_ip,registered_ip,paired_ip,processed,purchased_by_user_id,note,exported,first_accessed,ignore_first_accessed,title
0,794965,20141029u305,0,2014-10-29 21:10:47,2016-02-23 22:57:37,2014-10-29 21:10:47,0,2014-10-29 21:13:56,"{""tessitura_spoofing"":1,""ticket_id"":""794965"",""performance_id"":""71"",""attended"":""1"",""date_attended"":""2014-10-29T17:10:47-04:00""}",2014-10-29 23:19:52,0,0,0,0,0,NaN,NaN,NaT,1.0,NaN
1,795317,foo001,0,2014-10-29 23:32:29,2016-02-23 22:57:37,2014-10-29 23:32:29,0,2014-10-29 23:33:06,"{""tessitura_spoofing"":1,""ticket_id"":""795317"",""performance_id"":""71"",""attended"":""1"",""date_attended"":""2014-10-29T19:32:29-04:00""}",2014-10-29 23:35:18,0,0,0,0,0,NaN,NaN,NaT,1.0,NaN
2,795341,foo002,0,2014-10-29 23:40:51,2016-02-23 22:57:37,2014-10-29 23:40:51,0,2014-10-29 23:41:08,"{""tessitura_spoofing"":1,""ticket_id"":""795341"",""performance_id"":""71"",""attended"":""1"",""date_attended"":""2014-10-29T19:40:51-04:00""}",2014-10-29 23:41:21,0,0,0,0,0,NaN,NaN,NaT,1.0,NaN
3,795351,foo003,0,2014-10-29 23:43:31,2016-02-23 22:57:37,2014-10-29 23:43:31,0,2014-10-29 23:44:28,"{""tessitura_spoofing"":1,""ticket_id"":""795351"",""performance_id"":""71"",""attended"":""1"",""date_attended"":""2014-10-29T19:43:31-04:00""}",2014-10-29 23:44:34,0,0,0,0,0,NaN,NaN,NaT,1.0,NaN
4,795405,foo004,0,2014-10-30 00:13:47,2016-02-23 22:57:37,2014-10-30 00:13:47,0,2014-10-30 00:14:13,"{""tessitura_spoofing"":1,""ticket_id"":""795405"",""performance_id"":""71"",""attended"":""1"",""date_attended"":""2014-10-29T20:13:47-04:00""}",2014-10-30 00:14:41,0,0,0,0,0,NaN,NaN,NaT,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
716277,132806719,56bs6,0,2020-07-09 13:11:50,2020-07-09 13:22:11,2020-07-09 13:12:12,0,NaT,"{""status"":""Attendance recorded for ticket: 940560"",""ticket_id"":""940560"",""shortcode"":""56bs6"",""performance_id"":""4254"",""performance_date"":""2020-07-09T10:00:00-04:00"",""date_attended"":""Jul 9 2020 9:12AM"",""attended"":""1"",""type"":null}",NaT,1198074958,1198074958,0,0,0,NaN,NaN,2020-07-09 13:22:11,NaN,NaN
716278,132806727,cvszd,0,2020-07-09 13:16:19,2020-07-09 13:22:42,2020-07-09 13:16:32,0,NaT,"{""status"":""Attendance recorded for ticket: 940561"",""ticket_id"":""940561"",""shortcode"":""cvszd"",""performance_id"":""4254"",""performance_date"":""2020-07-09T10:00:00-04:00"",""date_attended"":""Jul 9 2020 9:16AM"",""attended"":""1"",""type"":null}",NaT,1198074958,1198074958,0,0,0,NaN,NaN,2020-07-09 13:22:42,NaN,NaN
716279,132806735,zrbqg,0,2020-07-09 13:19:41,2020-07-09 13:22:54,2020-07-09 13:19:58,0,NaT,"{""status"":""Attendance recorded for ticket: 940562"",""ticket_id"":""940562"",""shortcode"":""zrbqg"",""performance_id"":""4254"",""performance_date"":""2020-07-09T10:00:00-04:00"",""date_attended"":""Jul 9 2020 9:19AM"",""attended"":""1"",""type"":null}",NaT,1198074958,1198074958,0,0,0,NaN,NaN,2020-07-09 13:22:54,NaN,NaN
716280,132806749,Jr656,0,2020-07-09 13:40:43,2020-07-09 13:40:43,2020-07-09 13:40:42,0,NaT,"{""status"":""Attendance recorded for ticket: 940563"",""ticket_id"":""940563"",""shortcode"":""Jr656"",""performance_id"":""4254"",""performance_date"":""2020-07-09T10:00:00-04:00"",""date_attended"":""Jul 9 2020 9:40AM"",""attended"":""1"",""type"":null}",NaT,1198074958,1198074958,0,0,0,NaN,NaN,NaT,NaN,NaN


In [30]:
visit_accessed = visits[['created', 'first_accessed', 'ignore_first_accessed']]

In [34]:
visit_accessed = visit_accessed[visit_accessed['ignore_first_accessed'].isna()]

In [35]:
visit_accessed['time_between'] = visit_accessed['first_accessed'] - visit_accessed['created']

/Users/shirin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [36]:
visit_accessed

,created,first_accessed,ignore_first_accessed,time_between
144641,2016-02-24 14:46:38,NaT,NaN,NaT
144642,2016-02-24 14:55:22,NaT,NaN,NaT
144643,2016-02-24 14:55:58,NaT,NaN,NaT
144644,2016-02-24 14:56:38,NaT,NaN,NaT
144645,2016-02-24 14:57:33,2016-02-25 00:37:53,NaN,09:40:20
...,...,...,...,...
716277,2020-07-09 13:11:50,2020-07-09 13:22:11,NaN,00:10:21
716278,2020-07-09 13:16:19,2020-07-09 13:22:42,NaN,00:06:23
716279,2020-07-09 13:19:41,2020-07-09 13:22:54,NaN,00:03:13
716280,2020-07-09 13:40:43,NaT,NaN,NaT


In [37]:
visit_accessed['time_between'].mean()
#~7.5 weeks
#need to keep exploring the post_engagemnt: if the first_access is absolute true, 
#why the values when claculating first logs in created and bundle not giving the same

Timedelta('53 days 00:27:39.603057')

In [47]:
visit_accessed_notna = visit_accessed[visit_accessed['first_accessed'].notna()]

In [48]:
visit_accessed_notna['first_accessed'].count()

114616

In [49]:
visit_accessed_notna['first_accessed'].count() / visits['created'].count() * 100
#perc of engagment

16.001518954825055

In [ ]:
#needs to keep explore the first access, and why the relation of bundles is not 
#giving the same relation. something to talk through with someone.. 